In [1]:
import pandas as pd
import numpy as np

In [2]:
import xarray as xr

In [3]:
from datetime import datetime
import time, calendar
import pandas as pd

from src.config import *
from src.constants import *
from src.data.poloniex import Poloniex

In [4]:
start = datetime(2020, 1, 20, 3, 10)
start = int(time.mktime(start.timetuple()) - time.timezone)
pd.to_datetime(start + HALF_HOUR - start%HALF_HOUR, unit = 's')

Timestamp('2020-01-20 03:30:00')

In [5]:
polo = Poloniex()

start = datetime(2020, 1, 20, 3, 0)
end= datetime(2020, 1, 20, 4, 20)

start = int(time.mktime(start.timetuple()) - time.timezone)
end= int(calendar.timegm(end.timetuple()))

period = FIFTEEN_MINUTES

In [6]:
from src.data.globaldatamatrix import HistoryManager
mgr = HistoryManager(coin_number=4, end = end)

In [7]:
panel = mgr.get_global_panel(start=start, end=end, period=period, features=['close', 'high', 'low'])

In [8]:
coin = panel.coins[0].values

In [9]:
import sqlite3
conn = sqlite3.connect(DATABASE_DIR)
cursor = conn.cursor()
cursor.execute("select * from History")
df = pd.DataFrame(cursor.fetchall())
df[0] = pd.to_datetime(df[0], unit='s')
df.columns = [d[0] for d in cursor.description]
conn.close()
df = df[df.coin == coin]

In [10]:
p = panel.sel(coins=coin, features='high')
p.values

array([0.00011567, 0.00011567, 0.00011571, 0.00011567, 0.00011569,
       0.0001155 ])

In [11]:
df['highh'] = df['high'].rolling(3).max()

In [12]:
db = df[((df.date.astype(int)/10**9).apply(int))%FIFTEEN_MINUTES==0]
db = db.highh.values

In [13]:
db

array([       nan, 0.00011567, 0.00011571, 0.00011567, 0.00011569,
       0.00011558])

In [20]:
db==p.values

array([False,  True,  True, False,  True, False])

In [49]:
conn = sqlite3.connect(DATABASE_DIR)
sql = ("SELECT date_norm, MAX(high)" +
                               " FROM (SELECT date+{period}-(date%{period})"
                               " AS date_norm, high, coin FROM History)"
                               " WHERE date_norm>={start} and date_norm<={end} and coin=\"{coin}\""
                               " GROUP BY date_norm".format(
                                    period=period,start=start,end=end,coin=coin))

sql = ("SELECT date+{period}-(date%{period})" 
       "AS date_norm, high, coin FROM History".format(period=period))

q = pd.read_sql_query(sql, 
                  con=conn, 
                  parse_dates=["date_norm"]) #, 
         #         index_col="date_norm")
q[:17]

,date_norm,high,coin
0,2020-01-20 03:15:00,0.000116,reversed_USDT
1,2020-01-20 03:15:00,0.000116,reversed_USDT
2,2020-01-20 03:15:00,0.000116,reversed_USDT
3,2020-01-20 03:30:00,0.000116,reversed_USDT
4,2020-01-20 03:30:00,0.000116,reversed_USDT
5,2020-01-20 03:30:00,0.000116,reversed_USDT
6,2020-01-20 03:45:00,0.000116,reversed_USDT
7,2020-01-20 03:45:00,0.000116,reversed_USDT
8,2020-01-20 03:45:00,0.000116,reversed_USDT
9,2020-01-20 04:00:00,0.000116,reversed_USDT


In [52]:
sql = ("SELECT * FROM History WHERE coin=\"{coin}\"".format(coin=coin))
q = pd.read_sql_query(sql, 
                  con=conn, 
                  parse_dates=["date"]) #, 
         #         index_col="date_norm")
q

,date,coin,high,low,open,close,volume,quoteVolume,weightedAverage
0,2020-01-20 03:00:00,reversed_USDT,0.000116,0.000115,0.000115,0.000116,0.723622,6265.847586,0.000115
1,2020-01-20 03:05:00,reversed_USDT,0.000116,0.000115,0.000115,0.000116,3.003744,25994.398273,0.000116
2,2020-01-20 03:10:00,reversed_USDT,0.000116,0.000116,0.000116,0.000116,10.791788,93362.643500,0.000116
3,2020-01-20 03:15:00,reversed_USDT,0.000116,0.000115,0.000116,0.000116,5.631459,48761.975858,0.000115
4,2020-01-20 03:20:00,reversed_USDT,0.000116,0.000115,0.000116,0.000116,15.775775,136448.572905,0.000116
5,2020-01-20 03:25:00,reversed_USDT,0.000116,0.000116,0.000116,0.000116,5.712826,49398.503834,0.000116
6,2020-01-20 03:30:00,reversed_USDT,0.000116,0.000116,0.000116,0.000116,2.319727,20072.264372,0.000116
7,2020-01-20 03:35:00,reversed_USDT,0.000116,0.000116,0.000116,0.000116,3.357451,29037.488844,0.000116
8,2020-01-20 03:40:00,reversed_USDT,0.000116,0.000116,0.000116,0.000116,2.055997,17785.264681,0.000116
9,2020-01-20 03:45:00,reversed_USDT,0.000116,0.000116,0.000116,0.000116,7.686406,66480.261137,0.000116


In [53]:
time_index = pd.to_datetime(list(range(start, end+1, FIFTEEN_MINUTES)),unit='s')
time_index


DatetimeIndex(['2020-01-20 03:00:00', '2020-01-20 03:15:00',
               '2020-01-20 03:30:00', '2020-01-20 03:45:00',
               '2020-01-20 04:00:00', '2020-01-20 04:15:00'],
              dtype='datetime64[ns]', freq=None)

In [ ]:
# for coin in mgr.select_coins(start=start , end=end):
#     mgr.update_data(start=start, end=end, coin=coin)

In [ ]:
import sqlite3
conn = sqlite3.connect(DATABASE_DIR)
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

In [ ]:
coin = mgr.select_coins(start, end)[0]
print(coin)
sqlll = ("SELECT date+300 AS date_norm, close FROM History WHERE"
                               " date_norm>={start} and date_norm<={end}" 
                               " and date_norm%{period}=0 and coin=\"{coin}\"".format(   # Should always give date_norm%period=0 as we subtract the residual from the input dates
                               start=start, end=end, period=period, coin=coin))
sql = ("SELECT date+{period} AS date_norm, open FROM History WHERE"
                               " date_norm>={start} and date_norm<={end}" 
                               " and date_norm%{period}=0 and coin=\"{coin}\"".format(
                               start=start, end=end, period=period, coin=coin))

serial_data = pd.read_sql_query(sql, con=conn,
                                                    parse_dates=["date_norm"],
                                                    index_col="date_norm")
serial_data

In [ ]:
cursor.execute("SELECT * from History WHERE coin=\"{coin}\"".format(coin=coin))
df = pd.DataFrame(data = cursor.fetchall())
df[0] = pd.to_datetime(df[0], unit = 's')
df

In [ ]:
chart = polo.marketChart(period=FIVE_MINUTES, start=start, end=end, pair = 'BTC_' + coin)
chart = pd.DataFrame(chart)
chart.date = pd.to_datetime(chart.date, unit = 's')
chart

In [ ]:
mgr.select_coins(start,end)

In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
class Cash(tf.keras.Model):

    def __init__(self):
        super(Cash, self).__init__()

        self.dense = tf.keras.layers.Dense(5, activation='relu')
 
        self.b = tf.Variable(tf.zeros((1, 1), dtype=tf.float32), trainable=True)
        self.out = tf.keras.layers.Activation('softmax')

    def call(self, x):
        x = self.dense(x)
        bias = tf.tile(self.b, [tf.shape(x)[0], 1])
        x = tf.concat((bias, x), axis = -1)
        x = self.out(x)
        
        return x
    
    
model = Cash()

X = np.random.randn(100, 3)
with tf.GradientTape() as tape:
    y = model(X)

print([var.name for var in tape.watched_variables()])
grads = tape.gradient(y, model.trainable_variables)

In [ ]:
class Cash(tf.keras.Model):

    def __init__(self, rows = 11, cols = 50, features = 3):
        super(Cash, self).__init__()
        
        input_shape = (rows, cols, features)
        inputs = tf.keras.Input(shape=input_shape)
        x = tf.keras.layers.Conv2D(
            filters = 2, 
            kernel_size = (1,3), 
            padding='valid', 
            activation='relu',
            name = 'conv1'
        )(inputs)
        x = tf.keras.layers.Conv2D(1, 
                                (1, x.shape[2]), 
                                activation="relu", 
                                name = 'conv2')(x)
        x = tf.squeeze(x)
        
        self.model = tf.keras.Model(inputs = inputs, outputs = x)
        
        self.b = tf.Variable(tf.zeros((1, 1), dtype=tf.float32), trainable=True)
        self.out = tf.keras.layers.Activation('softmax')

    def call(self, x):
        x = self.model(x)
        print(x.shape)
        bias = tf.tile(self.b, [tf.shape(x)[0], 1])
        x = tf.concat((bias, x), axis = -1)
        x = self.out(x)
        
        return x
    
    
model = Cash()

X = np.random.randn(100, 11, 50, 3)
with tf.GradientTape() as tape:
    y = model(X)

print([var.name for var in tape.watched_variables()])
grads = tape.gradient(y, model.trainable_variables)

In [ ]:
batch = 100
dense = tf.keras.layers.Dense(5, activation='relu')
b = tf.Variable(tf.zeros((1,1), dtype=tf.float32), name='b', trainable=True)

x =  tf.random.normal((batch, 3))
with tf.GradientTape() as tape:
    y = dense(x)
    cash = tf.tile(b, [y.shape[0], 1])
    y = tf.concat((cash, y), axis = -1)
    y = tf.keras.layers.Activation('softmax')(y)

print([var.name for var in tape.watched_variables()])
grads = tape.gradient(y, [b, dense.trainable_variables])
grads

In [ ]:
def CNN(rows, cols, features, batch_size):
    input_shape = (rows, cols, features)
    X = keras.Input(shape= input_shape, batch_size=batch_size)
    w = keras.Input(shape = (rows, 1, 1), batch_size=batch_size)

    x = tf.keras.layers.Conv2D(
        filters = 2, 
        kernel_size = (1,3), 
        padding='valid', 
        activation='relu'
    )(X)
        
    x = keras.layers.Conv2D(20, 
                            (1, x.shape[2]), 
                            activation="relu", 
                            name = 'conv2')(x)  
    
    con = keras.layers.Concatenate(axis=3)([x, w])

    x = keras.layers.Conv2D(1, (1,1), name = 'votes')(con)
    x = tf.squeeze(x)
    
    b = tf.tile(b, [x.shape[0], 1])
    with_bias = keras.layers.Concatenate(axis=1)([b, x])
    #with_bias = CashBias()(x)

    outputs = keras.layers.Activation('softmax')(with_bias)
    return keras.Model(inputs = [X, w], outputs = outputs, name = "Policy")

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

In [ ]:
inputs = keras.Input(shape=(784,), name="digits")
x1 = layers.Dense(64, activation="relu")(inputs)
x2 = layers.Dense(64, activation="relu")(x1)
outputs = layers.Dense(10, name="predictions")(x2)
model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
# Instantiate an optimizer.
optimizer = keras.optimizers.SGD(learning_rate=1e-3)
# Instantiate a loss function.
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
# Prepare the training dataset.
batch_size = 64
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = np.reshape(x_train, (-1, 784))
x_test = np.reshape(x_train, (-1, 784))
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

In [ ]:
for (x, y) in train_dataset:

    with tf.GradientTape() as tape:

        # Run the forward pass of the layer.
        # The operations that the layer applies
        # to its inputs are going to be recorded
        # on the GradientTape.
        logits = model(x, training=True)  # Logits for this minibatch

        # Compute the loss value for this minibatch.
        loss_value = loss_fn(y, logits)

    # Use the gradient tape to automatically retrieve
    # the gradients of the trainable variables with respect to the loss.
    grads = tape.gradient(loss_value, model.trainable_weights)

    # Run one step of gradient descent by updating
    # the value of the variables to minimize the loss.
    # optimizer.apply_gradients(zip(grads, model.trainable_weights))
    shapes = [grad.shape for grad in grads]
    print(shapes)
    print(model.trainable_variables)
    break

In [ ]:
a = [np.random.rand(2,3) for i in range(10)]

In [ ]:
np.array(a, dtype='float32').dtype